In [19]:
import os
import qiime2
import skbio

import numpy as np
import pandas as pd

from tempfile import mkdtemp
from qiime2.plugins import (demux, dada2, phylogeny, metadata, fragment_insertion, diversity, 
                            feature_table, feature_classifier, taxa, composition
                           )
from qiime2 import Artifact, Metadata, Visualization

In [4]:
# Create separate directories for results to avoid confusion
art_dir = "./artifacts/"
vis_dir = "./visualizations/"
res_dir = "./results/"
tab_dir = "./tables/"

for name in ["artifacts", "visualizations", "results", "tables"]:
    os.makedirs(name, exist_ok=True)

In [4]:
workdir=''

In [5]:
!cd $workdir

This analysis is conducted using `QIIME2` version `2022.2.0`
# 1. Import raw demultiplexed data as a Qiime2 artifact
We receved already demultiplexed sequences from the lab, therefore we are skipping demultiplication and load data as a `QIIME2` artefact.

In [ ]:
merged_sequences = qiime2.Artifact.import_data('SampleData[SequencesWithQuality]', 
                                               "./dada2-sinus-manifest-file.csv,  
                                               view_type='PairedEndFastqManifestPhred33V2')

In [ ]:
merged_sequences.save(os.path.join(art_dir, 'paired_end_sequences.qza'))

In [5]:
merged_sequences = Artifact.load(os.path.join(art_dir, 'paired_end_sequences.qza'))

# 2. Demultiplexing summary
Next, we run `demux.summary`. It allows us to access per base quality of sequenencing.


In [6]:
demux_summary = demux.visualizers.summarize(merged_sequences)

In [7]:
demux_summary.visualization

<visualization: Visualization uuid: 00780854-86c3-4eae-801e-b9cef2140824>

In [11]:
demux_summary.visualization.save(os.path.join(vis_dir, 'demux.qzv')) #Visulization: Summarize counts per sample for all samples, and generate interactive
                                                                     #positional quality plots based on `n` randomly selected sequences.

'./visualizations/demux.qzv'

# 3. DADA2

After demultiplication we correct Illumina sequencing errors using `DADA2` and trim low-quality nucleotides. 

In [7]:
paired_end_sequences = Artifact.load(art_dir+'merged_sequences.qza')

In [9]:
paired_end_sequencesised_sequences = dada2.methods.denoise_paired(paired_end_sequences, trim_left_f = 0, trim_left_r = 0,\
trunc_len_f = 290, trunc_len_r = 250, n_threads = 16, min_fold_parent_over_abundance=4)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_paired.R /tmp/tmprkt018m9/forward /tmp/tmprkt018m9/reverse /tmp/tmprkt018m9/output.tsv.biom /tmp/tmprkt018m9/track.tsv /tmp/tmprkt018m9/filt_f /tmp/tmprkt018m9/filt_r 290 250 0 0 2.0 2.0 2 12 independent consensus 4 16 1000000



In [10]:
dada2_denoised_sequences.table.save(art_dir+'dada2_table.qza')
dada2_denoised_sequences.representative_sequences.save(art_dir+'dada2_rep_seqs.qza')
dada2_denoised_sequences.denoising_stats.save(art_dir+'dada2_stats.qza')

'./artifacts/dada2_stats.qza'

In [11]:
dada2_denoise_stats = metadata.visualizers.tabulate(Artifact.load(art_dir+"dada2_stats.qza").view(qiime2.Metadata))
dada2_denoise_stats.visualization

<visualization: Visualization uuid: e8b7db0c-cd29-4ad8-aa65-2a6d2a97b2db>

In [12]:
dada2_denoise_stats.visualization.save(vis_dir+'dada2_denoise_stats.qzv')

'./visualizations/dada2_denoise_stats.qzv'

## Visualizing `FeatureTable` summary
We visualize summary of feature table to get an idea of feature counts distribution in a study and estimate good cut-off for a rarefaction procedure. 

In [14]:
table = Artifact.load(art_dir+"dada2_table.qza")

In [15]:
summ = feature_table.visualizers.summarize(table)
summ.visualization

<visualization: Visualization uuid: bcc6726a-9e9f-42a5-888c-1f20b04f4935>

In [36]:
summ.visualization.save(vis_dir+"table-summary.qzv")

'./visualizations/table-summary.qzv'

# 4. Building phylogenetic tree using `q2-fragment-insertion`

In order to access phylogenetic metrices we need to build a phylogenetic tree. We use a SEPP fragment insertion methods, as it is the method that resolves the problems with *de novo* method and yields an additional benefit working on sOTU level.

In [25]:
# Downloading reference data from SILVA 
!wget https://data.qiime2.org/2021.4/common/sepp-refs-silva-128.qza

--2021-09-02 12:35:49--  https://data.qiime2.org/2021.4/common/sepp-refs-silva-128.qza
Translacja data.qiime2.org (data.qiime2.org)... 54.200.1.12
Łączenie się z data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 302 FOUND
Lokalizacja: https://s3-us-west-2.amazonaws.com/qiime2-data/2021.4/common/sepp-refs-silva-128.qza [podążanie]
--2021-09-02 12:35:51--  https://s3-us-west-2.amazonaws.com/qiime2-data/2021.4/common/sepp-refs-silva-128.qza
Translacja s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.169.176
Łączenie się z s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.169.176|:443... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 200 OK
Długość: 181253322 (173M) [binary/octet-stream]
Zapis do: `sepp-refs-silva-128.qza'

sepp-refs-silva-128 100%[===================>] 172,86M   920KB/s    w 3m 26s   

2021-09-02 12:39:18 (858 KB/s) - zapisano `sepp-refs-silva-128.qza' [181253322/1

In [ ]:
rooted_tree = fragment_insertion.methods.sepp(representative_sequences = Artifact.load(art_dir+"dada2_rep_seqs.qza"),
                                              reference_database = Artifact.load("sepp-refs-silva-128.qza"),
                                              threads=20)

In [ ]:
rooted_tree.tree.save(art_dir+"SEPP_tree_rooted.qza")

# 5.Bulding alpha-rarefaction curve
To normalize samples and account for different library sizes we use rarefaction. To better undestrand the behaviour of samples under different cut-off thresholds we build a rarefaction curve using `q2-diversity`.  
Our analysis is strongly intrested in microbiome composition within **different sinuses**, therefore we will rarefy with respect to these patient metadata.
To achieve this we need to load metadata into the `qiime2.Metdata` object.

In [16]:
# Format metadata from QIITA for use in QIIME2
tmp_meta = pd.read_csv("sinus_metadata_merged.txt", 
                                       na_values=["not applicable", "not collected"], 
                                       sep="\t", index_col=0).drop(columns=["dna_extracted",
                                                                            "physical_specimen_remaining"
                                                                           ]).to_csv("processed_metdata.tsv", sep="\t")

sample_metadata = Metadata.load("processed_metdata.tsv",
                                column_types={"host_subject_id" : "categorical"})

In [17]:
# Loading a tree file
tree = Artifact.load(art_dir+"SEPP_tree_rooted.qza")

In [ ]:
# Building a rarefaction curve to better estimate a rarefaction cut-off
rar = diversity.visualizers.alpha_rarefaction(table=table,
                                              phylogeny=tree, 
                                              metadata=sample_metadata,
                                              min_depth=4000, 
                                              max_depth=40000,
                                              steps=90, 
                                              iterations=20)

In [ ]:
rar.visualization.save(vis_dir+"alpha_rarefaction.qzv")

In [20]:
vis = Visualization.load(vis_dir+"alpha_rarefaction.qzv")

In [21]:
vis

<visualization: Visualization uuid: e179184c-b9cd-496b-8023-3d3999b9c5ee>

# 6. Rarefaction of samples

After studying the rarefaction curve we proceed with rarefaction of the samples.

In [13]:
rarefied = feature_table.methods.rarefy(table=table, 
                                        sampling_depth=20504,
                                        with_replacement=False)

In [14]:
rarefied.rarefied_table.save(art_dir+"dada2_table_rar_20504.qza")

'./artifacts/dada2_table_rar_20504.qza'

# 7. Beta-diversity analysis

In [22]:
rar_table = Artifact.load(art_dir+"dada2_table_rar_20504.qza")

In [29]:
# getting different distance matrices
# Bray-Curtis
braycurtis = diversity.pipelines.beta(rar_table, metric="braycurtis").distance_matrix
braycurtis.save(art_dir+"braycurtis_dis_matrix" + ".qza")
# Weighted UniFrac
wuni = diversity.pipelines.beta_phylogenetic(rar_table, metric="weighted_unifrac", 
                                             phylogeny=tree).distance_matrix
wuni.save(art_dir+"wuni_dis_matrix" + ".qza")

'./artifacts/wuni_dis_matrix.qza'

## 7.1. Comparison of different sampling sites - `host_body_site`

In [32]:
permanova = diversity.visualizers.beta_group_significance(distance_matrix=Artifact.load(art_dir + "braycurtis_dis_matrix" + ".qza"), 
                                                          metadata=sample_metadata.get_column("host_body_site"),
                                                          method="permanova", 
                                                          pairwise=True, 
                                                          permutations=9999)
permanova.visualization

In [34]:
permanova.visualization.save(vis_dir + "permanova_host_body_site_bc.qzv")

'./visualizations/permanova_host_body_site_bc.qzv'

In [35]:
permanova = diversity.visualizers.beta_group_significance(distance_matrix=Artifact.load(art_dir + "wuni_dis_matrix" + ".qza"), 
                                                          metadata=sample_metadata.get_column("host_body_site"),
                                                          method="permanova", 
                                                          pairwise=True, 
                                                          permutations=9999)
permanova.visualization

<visualization: Visualization uuid: 3b2e67a6-0b59-4391-a008-2c3436f5374f>

In [36]:
permanova.visualization.save(vis_dir + "permanova_host_body_site_wuni.qzv")

'./visualizations/permanova_host_body_site_wuni.qzv'

## 7.2 Comparison of different ostium conditions - `maxillary_ostium_patency`

In [38]:
permanova = diversity.visualizers.beta_group_significance(distance_matrix=Artifact.load(art_dir + "braycurtis_dis_matrix" + ".qza"), 
                                                          metadata=sample_metadata.get_column("maxillary_ostium_size"),
                                                          method="permanova", 
                                                          pairwise=True, 
                                                          permutations=9999)
permanova.visualization

<visualization: Visualization uuid: 5a67c8af-4506-4f47-acab-938813d0346e>

In [40]:
permanova.visualization.save(vis_dir + "permanova_maxillary_ostium_size_bc.qzv")

'./visualizations/permanova_maxillary_ostium_size_bc.qzv'

In [41]:
permanova = diversity.visualizers.beta_group_significance(distance_matrix=Artifact.load(art_dir + "wuni_dis_matrix" + ".qza"), 
                                                          metadata=sample_metadata.get_column("maxillary_ostium_size"),
                                                          method="permanova", 
                                                          pairwise=True, 
                                                          permutations=9999)
permanova.visualization

<visualization: Visualization uuid: d86b2f9b-beee-441a-a063-0bbce97fd568>

In [43]:
permanova.visualization.save(vis_dir + "permanova_maxillary_ostium_size_bc.qzv")

'permanova_maxillary_ostium_size_bc.qzv'

## 7.3. Adonis

In [45]:
# ADONIS sample_id tweaking due to R error
# described in detail https://forum.qiime2.org/t/vegan-error-in-diversity-adonis-qiime2-api/20635
meta = pd.read_csv("sinus_metadata_merged.txt", sep="\t", index_col=0)
meta.columns = meta.columns.str.replace(".", "_", regex=True)
meta.index = meta.index.str.replace(".", "_", regex=True)
meta = meta.applymap(lambda x: x.strip() if isinstance(x, str) else x)
adonis_metadata = Metadata(meta)

filename = "braycurtis_dis_matrix.qza"
dis_matrix = Artifact.load(art_dir+filename).view(skbio.DistanceMatrix).to_data_frame()
dis_matrix.columns = dis_matrix.columns.str.replace(".", "_", regex=True)
dis_matrix.index = dis_matrix.index.str.replace(".", "_", regex=True) 
dis_art = Artifact.import_data("DistanceMatrix", skbio.DistanceMatrix(dis_matrix, ids=dis_matrix.index))
ad = diversity.visualizers.adonis(dis_art, 
                                  metadata=adonis_metadata,
                                  formula="host_body_site * host_subject_id",
                                  permutations=9999
                                  )
ad.visualization

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: run_adonis.R /tmp/tmptmnsvcb3/dm.tsv /tmp/tmptmnsvcb3/md.tsv host_body_site * host_subject_id 9999 1 /tmp/qiime2-temp-z5vq6t8x/adonis.tsv

R version 4.1.3 (2022-03-10) 


Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


<visualization: Visualization uuid: 5c012dff-ce81-4121-a613-590f9fdc80c2>

In [50]:
ad.visualization.save(vis_dir + "adonis_bc.qzv")

'./visualizations/adonis_bc.qzv'

# 8. Taxonomical analysis
To classify reads and assign taxonomy to them we used a classifier trained with or region of interes using `RESCRIPt`.   
 A detailed tutorial on how to do this may be found on [QIIME2 Forum](https://forum.qiime2.org/t/processing-filtering-and-evaluating-the-silva-database-and-other-reference-sequence-data-with-rescript/15494).


In [64]:
tax = feature_classifier.methods.classify_sklearn(reads = rep_seqs, 
                                                  classifier = Artifact.load("./artifacts/silva-138-ssu-nr99-reg-of-int-classifier.qza"),
                                                  n_jobs=-2)

In [65]:
tax.classification.save(art_dir + "tax-reg-of-int.qza")

'./artifacts/tax-reg-of-int.qza'

In [68]:
barp = taxa.visualizers.barplot(table=Artifact.load(art_dir + "dada2_table_rar_20504.qza"),
                                taxonomy = Artifact.load(art_dir + "tax-reg-of-int.qza"), 
                                metadata=sample_metadata)

In [69]:
barp.visualization.save(vis_dir+"taxa_barplot_reg_of_int.qzv")

'./visualizations/taxa_barplot_reg_of_int.qzv'

In [71]:
Visualization.load(vis_dir + "taxa_barplot_reg_of_int.qzv")

<visualization: Visualization uuid: 8b63669c-95c8-4083-a07a-5d3f3af515d8>

# 9. ANCOM analysis
We conduct a differential abundance testing in order to see if certain ASVs differ in abundance with respect to ostium sinus closure.   
It was not a straightforward operation, as we should have filtered `null` values from metadata and then remove features with `<10` counts in order to make computational time reasonable.
The issue resulted in improvement of `ANCOM` behaviour described in [GitHub issue](https://github.com/qiime2/q2-composition/issues/90).

In [51]:
table = Artifact.load(art_dir+"dada2_table_rar_20504.qza")
metadata_col = "maxillary_ostium_size"

In [52]:
ancom_meta = sample_metadata.get_column(metadata_col)

In [53]:
ancom_table = feature_table.methods.filter_samples(table=table,
                                                    metadata=sample_metadata, 
                                                    where=f"[{metadata_col}] NOT NULL",
                                                   ).filtered_table

ancom_table = taxa.methods.collapse(table=ancom_table,
                                    taxonomy=Artifact.load(art_dir + "tax-reg-of-int.qza"),
                                    level=7).collapsed_table

ancom_table = feature_table.methods.filter_features(table=ancom_table,
                                                    min_frequency=10).filtered_table

ancom_table = composition.methods.add_pseudocount(table=ancom_table).composition_table

In [54]:
composition.visualizers.ancom(table = ancom_table, 
                              metadata = ancom_meta, 
                              transform_function = "clr",
                              ).visualization.save(vis_dir+"ancom_" + metadata_col + ".qzv")

/home/MCB/vbezshapkin/miniconda3/envs/qiime2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


'./visualizations/ancom_maxillary_ostium_size.qzv'

In [55]:
Visualization.load(vis_dir+"ancom_" + metadata_col + ".qzv")

<visualization: Visualization uuid: 3544899e-6d2a-42cc-9ca3-0883a4400053>

However, no significantly different ASVs were found. 

In [4]:
frequency_taxonomy = taxa.methods.collapse(table=Artifact.load(art_dir + "dada2_table_rar_20504.qza"),
                                           taxonomy=Artifact.load(art_dir + "tax-reg-of-int.qza"),
                                           level=7)  # Species level

In [5]:
frequency_taxonomy.collapsed_table.save(art_dir+'collapsed_table.qza')

'./artifacts/collapsed_table.qza'

In [18]:
watermark -iv

qiime2: 2022.2.0
skbio : 0.5.6
numpy : 1.22.3
pandas: 1.2.5

